In [1]:
# Import necessary libraries for data manipulation, plotting, and network analysis
import networkx as nx  # For handling graph data structures
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation using DataFrames
import logging  # For logging messages
import random  # For generating random numbers
import warnings
import os
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import json

#import GridCalEngine.api as gce  # For interfacing with the GridCal API
#import GridCalEngine.Devices as dev
#import GridCalEngine.Simulations as sim
#from GridCalEngine.Compilers.circuit_to_newton_pa import translate_newton_pa_pf_results, newton_pa_pf
#from GridCalEngine.IO.file_handler import FileOpen
#import GridCalEngine.enumerations as en

import VeraGridEngine.api as gce  # For interfacing with the GridCal API
import VeraGridEngine.Devices as dev
import VeraGridEngine.Simulations as sim
from VeraGridEngine.Compilers.circuit_to_newton_pa import translate_newton_pa_pf_results, newton_pa_pf
from VeraGridEngine.IO.file_handler import FileOpen
import VeraGridEngine.enumerations as en

import pandapower as pp
import simbench as sb
import pandapower.topology as top  # For topology analysis in Pandapower
import pandapower.plotting as plot  # For plotting in Pandapower
import pandapower.networks as nw

import src.GC_PandaPowerImporter as GC_PandaPowerImporter

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

In [2]:
sb_code1 = "1-HVMV-urban-2.203-0-no_sw"
gridPP = sb.get_simbench_net(sb_code1)
gridPP.switch.drop([232,234,236,238,240, 242,244,246], inplace=True)
gridPP.ext_grid.at[0,'name']="grid_ext"
gridPP.line['in_service'] = True

#print(gridPP.ext_grid)
#print(gridPP.bus.loc[gridPP.ext_grid.bus])

#pp.topology.create_nxgraph(gridPP)
#print(pp.topology.connected_components(gridPP))

#print(gridPP.switch.head())
#print(len(gridPP.switch))

#pp.diagnostic(net=gridPP)

pp.runpp(gridPP)
print("powerflow of the original pandapower network in pandapower")
print("   bus -5: ", gridPP.res_bus.tail(4))
print("   line -5: ", gridPP.res_line.tail(1))
print("   power losses:", gridPP.res_line.pl_mw.sum(), gridPP.res_line.ql_mvar.sum())

gridGC = GC_PandaPowerImporter.PP2GC(gridPP)

for line in gridGC.lines:
    line.active = True
options = gce.PowerFlowOptions(gce.SolverType.NR, initialize_with_existing_solution=False,control_q=False, verbose=False)
power_flowPP2GC = gce.PowerFlowDriver(gridGC, options)
power_flowPP2GC.run()
print("   ", power_flowPP2GC.results.get_bus_df().tail(4))
print("   ", power_flowPP2GC.results.get_branch_df().tail(1))
print("   power losses:", power_flowPP2GC.results.losses.sum())

LoadflowNotConverged: Power Flow nr did not converge after 10 iterations!

In [3]:
import pandapower as pp
import networkx as nx

# 1) make sure gridPP is loaded
G = pp.topology.create_nxgraph(gridPP)
cc = list(nx.connected_components(G))
print("Number of connected components:", len(cc))
for i, comp in enumerate(cc):
    print(f"component {i}: total buses = {len(comp)}; example buses = {sorted(list(comp))[:20]}")
# ext_grid location
ext_bus = int(gridPP.ext_grid.at[0,'bus'])
for i, comp in enumerate(cc):
    if ext_bus in comp:
        print("ext_grid (bus {}) is in component {}".format(ext_bus, i))
        break
else:
    print("ext_grid's bus not found in any component!")

# 2) check r/x/length is/not 0 or super small
lines = gridPP.line.copy()
rcol = 'r_ohm_per_km' if 'r_ohm_per_km' in lines.columns else 'r_ohm_per_km'
xcol = 'x_ohm_per_km' if 'x_ohm_per_km' in lines.columns else 'x_ohm_per_km'
lencol = 'length_km' if 'length_km' in lines.columns else 'length_km'

zero_imp = lines[(lines[rcol].fillna(0).abs() < 1e-12) & (lines[xcol].fillna(0).abs() < 1e-12)]
zero_len = lines[lines[lencol].fillna(1e9) == 0]
very_small = lines[(lines[rcol].fillna(0).abs() < 1e-6) & (lines[xcol].fillna(0).abs() < 1e-6)]

print("Lines with r==0 & x==0:", zero_imp.index.tolist())
print("Lines with length == 0:", zero_len.index.tolist())
print("Lines with very small r/x:", very_small.index.tolist()[:80])

# 3) check trafo para
print(gridPP.trafo[['vn_hv_kv','vn_lv_kv','vk_percent','vkr_percent']])
print("Transformers with vk_percent > 15:", gridPP.trafo[gridPP.trafo['vk_percent']>15].index.tolist())

# 4) check existance of lonely buses
connected_buses = set()
if 'from_bus' in gridPP.line.columns and 'to_bus' in gridPP.line.columns:
    connected_buses |= set(gridPP.line['from_bus'].unique())
    connected_buses |= set(gridPP.line['to_bus'].unique())
if 'hv_bus' in gridPP.trafo.columns:
    connected_buses |= set(gridPP.trafo['hv_bus'].unique())
if 'lv_bus' in gridPP.trafo.columns:
    connected_buses |= set(gridPP.trafo['lv_bus'].unique())

isolated = sorted(list(set(gridPP.bus.index) - connected_buses))
print("Number of isolated buses (no line/trafo connection):", len(isolated))
print("Some isolated buses:", isolated[:80])

# 5) list in_service=False line/trafo
print("Lines out-of-service count:", int((gridPP.line['in_service']==False).sum()))
print("Example lines out-of-service:", gridPP.line[gridPP.line['in_service']==False].index.tolist()[:40])
print("Trafos out-of-service count:", int((gridPP.trafo['in_service']==False).sum()))
print("Example trafos out-of-service:", gridPP.trafo[gridPP.trafo['in_service']==False].index.tolist()[:40])


Number of connected components: 1
component 0: total buses = 196; example buses = [np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20)]
ext_grid (bus 361) is in component 0
Lines with r==0 & x==0: []
Lines with length == 0: []
Lines with very small r/x: []
    vn_hv_kv   vn_lv_kv  vk_percent  vkr_percent
0 110.000000  20.000000   16.200000     0.340000
1 110.000000  20.000000   16.200000     0.340000
2 220.000000 110.000000   12.000000     0.128333
3 220.000000 110.000000   12.000000     0.128333
4 220.000000 110.000000   12.000000     0.128333
Transformers with vk_percent > 15: [0, 1]
Number of isolated buses (no line/trafo connection): 0
Some isolated buses: []
Lines out-of-service count: 0
Example lines out-of-service: []
Trafos out-of-service count: 0
Example trafos o

In [4]:
# 1) check injection
print("loads: count", len(gridPP.load), " total P MW:", gridPP.load['p_mw'].sum(), " min/max P_mw:", gridPP.load['p_mw'].min(), gridPP.load['p_mw'].max())
print(gridPP.load[['bus','p_mw','q_mvar']].sort_values('p_mw', ascending=False).head(10))

print("sgen: count", len(gridPP.sgen), " total P MW:", gridPP.sgen['p_mw'].sum(), " min/max P_mw:", gridPP.sgen['p_mw'].min(), gridPP.sgen['p_mw'].max())
print(gridPP.sgen[['bus','p_mw','q_mvar']].sort_values('p_mw', ascending=False).head(10))

print("gen: count", len(gridPP.gen), " total P MW:", gridPP.gen['p_mw'].sum() if len(gridPP.gen)>0 else 0)
print("ext_grid entries:", gridPP.ext_grid)

loads: count 194  total P MW: 521.7199999999999  min/max P_mw: 0.08 49.71
     bus      p_mw    q_mvar
193  560 49.710000 19.647000
192  448 49.710000 19.647000
181  526 34.480000 13.627000
182  528 34.480000 13.627000
183  643 34.480000 13.627000
189  451 31.640000 12.505000
190  675 31.640000 12.505000
191  655 31.640000 12.505000
186  569 17.260000  6.822000
184  424 17.260000  6.822000
sgen: count 219  total P MW: 609.9989999999999  min/max P_mw: 0.0 46.22
     bus      p_mw   q_mvar
215  451 46.220000 0.000000
208  414 33.480000 0.000000
217  619 33.480000 0.000000
190  424 25.570000 0.000000
192  569 25.570000 0.000000
193  669 25.570000 0.000000
191  461 25.570000 0.000000
194  712 25.570000 0.000000
212  586 25.070000 0.000000
196  675 23.800000 0.000000
gen: count 0  total P MW: 0
ext_grid entries:        name  bus    vm_pu  va_degree  slack_weight  in_service controllable voltLvl sn_mva min_q_mvar p_disp_mw profile max_p_mw max_q_mvar type    phys_type min_p_mw       subnet
0

In [5]:
# 2) test
import pandapower as pp

# try A: NR + DC init + larger iteration number
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("nr+dc converged")
except Exception as e:
    print("nr+dc failed:", repr(e))

# try B: BFSW
try:
    pp.runpp(gridPP, algorithm='bfsw')
    print("bfsw converged")
except Exception as e:
    print("bfsw failed:", repr(e))

nr+dc failed: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')
bfsw failed: ValueError('negative axis 1 index: -1')


In [6]:
# 3) temp change trafo: vk_percent
# backup rn value
orig_vk = gridPP.trafo['vk_percent'].copy()
# turn all vk_percent > 15 trafo to 15 (temp)
gridPP.trafo.loc[gridPP.trafo['vk_percent']>15, 'vk_percent'] = 15.0
print("changed vk_percent for trafos:", gridPP.trafo.loc[gridPP.trafo['vk_percent']==15].index.tolist())

# try again nr+dc
import pandapower as pp
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("nr+dc converged after lowering vk_percent")
except Exception as e:
    print("nr+dc still failed after lowering vk_percent:", repr(e))

# back to orginal value
gridPP.trafo['vk_percent'] = orig_vk

changed vk_percent for trafos: [0, 1]
nr+dc still failed after lowering vk_percent: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')


In [7]:
# 0 injection
# backup
loads_p = gridPP.load['p_mw'].copy()
loads_q = gridPP.load['q_mvar'].copy()
sgen_p  = gridPP.sgen['p_mw'].copy()
sgen_q  = gridPP.sgen['q_mvar'].copy()

# 0 all load and sgen
gridPP.load.loc[:, ['p_mw','q_mvar']] = 0.0
gridPP.sgen.loc[:, ['p_mw','q_mvar']] = 0.0

# run NR + dc init, bigger iter
import pandapower as pp
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("ZERO-INJECTION runpp: CONVERGED")
except Exception as e:
    print("ZERO-INJECTION runpp: FAILED ->", repr(e))

# recover
gridPP.load['p_mw'] = loads_p
gridPP.load['q_mvar'] = loads_q
gridPP.sgen['p_mw'] = sgen_p
gridPP.sgen['q_mvar'] = sgen_q

ZERO-INJECTION runpp: FAILED -> LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')


In [8]:
# Failed
print("lines r_ohm_per_km min/max:", gridPP.line['r_ohm_per_km'].min(), gridPP.line['r_ohm_per_km'].max())
print("lines x_ohm_per_km min/max:", gridPP.line['x_ohm_per_km'].min(), gridPP.line['x_ohm_per_km'].max())
print("lines length_km min/max:", gridPP.line['length_km'].min(), gridPP.line['length_km'].max())
# show extreme
print(gridPP.line[['from_bus','to_bus','r_ohm_per_km','x_ohm_per_km','length_km']].sort_values('r_ohm_per_km').head(10))
print(gridPP.line[['from_bus','to_bus','r_ohm_per_km','x_ohm_per_km','length_km']].sort_values('x_ohm_per_km').head(10))
print(gridPP.trafo[['hv_bus','lv_bus','vn_hv_kv','vn_lv_kv','vk_percent','vkr_percent','parallel']])
pairs = gridPP.line.groupby(['from_bus','to_bus']).size()
dup_pairs = pairs[pairs>1]
print("Duplicate (from,to) line pairs:", dup_pairs.head(20))

lines r_ohm_per_km min/max: 0.1095 0.5939
lines x_ohm_per_km min/max: 0.119 0.372
lines length_km min/max: 0.15 51.636
     from_bus  to_bus  r_ohm_per_km  x_ohm_per_km  length_km
121       388     537      0.109500      0.296000   4.682650
122       473     704      0.109500      0.296000   5.839610
123       473     704      0.109500      0.296000   5.839610
127       424     430      0.109500      0.296000   3.192820
220       398     586      0.109500      0.296000   7.133880
219       443     619      0.109500      0.296000   7.332100
217       560     455      0.109500      0.296000   7.122820
222       408     473      0.109500      0.296000   8.854110
216       661     678      0.109500      0.296000  11.166600
211       398     458      0.109500      0.296000  30.555000
     from_bus  to_bus  r_ohm_per_km  x_ohm_per_km  length_km
23         24      25      0.253000      0.119000   0.500000
24         25      26      0.253000      0.119000   0.350000
22          0      24      

In [9]:
# PLAN A 一次性临时禁用“重复对”中的额外线路与多余并联变压器（快速验证）
import pandapower as pp
from collections import defaultdict

# 备份
line_in_service_backup = gridPP.line['in_service'].copy()
trafo_in_service_backup = gridPP.trafo['in_service'].copy()

# 找到重复 (from_bus, to_bus) 对（只考虑有重复的）
pairs = gridPP.line.groupby(['from_bus','to_bus']).apply(lambda df: list(df.index)).to_dict()
dup_pairs = {k:v for k,v in pairs.items() if len(v)>1}

# 将每个重复对中，保留第一个，其他设为 in_service=False
disabled_lines = []
for k, idx_list in dup_pairs.items():
    for idx in idx_list[1:]:
        gridPP.line.at[idx, 'in_service'] = False
        disabled_lines.append(idx)

# 对 trafo：找 hv_bus/lv_bus 重复的（并联 trafo）
trafo_pairs = gridPP.trafo.groupby(['hv_bus','lv_bus']).apply(lambda df: list(df.index)).to_dict()
dup_trafos = {k:v for k,v in trafo_pairs.items() if len(v)>1}

disabled_trafos = []
for k, idx_list in dup_trafos.items():
    for idx in idx_list[1:]:
        gridPP.trafo.at[idx, 'in_service'] = False
        disabled_trafos.append(idx)

print("Temporarily disabled line indices (duplicates):", disabled_lines[:80])
print("Temporarily disabled trafo indices (duplicates):", disabled_trafos)

# 运行潮流
try:
    pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
    print("runpp succeeded AFTER disabling duplicate extra branches.")
    conv_success = True
except Exception as e:
    print("runpp STILL failed after disabling duplicates:", repr(e))
    conv_success = False

# 恢复原始 in_service 状态（无论成功与否）
gridPP.line['in_service'] = line_in_service_backup
gridPP.trafo['in_service'] = trafo_in_service_backup

print("Restored original in_service flags.")

Temporarily disabled line indices (duplicates): [233, 208, 227, 232, 140, 225, 231, 151, 186, 173, 123, 226, 187, 178, 170, 139, 166, 161, 228, 188, 229, 180, 223, 230]
Temporarily disabled trafo indices (duplicates): [3, 4, 1]
runpp STILL failed after disabling duplicates: LoadflowNotConverged('Power Flow nr did not converge after 200 iterations!')
Restored original in_service flags.


In [10]:
# PLAN B 若一次性禁用所有重复额外分支仍不收敛，逐对测试哪些具体重复对触发了问题。下面脚本会对每一对重复（线或 trafo）单独禁用它的额外条目并测试，输出每对测试结果，从而定位出“有问题”的具体对。
import pandapower as pp

# 备份
line_backup = gridPP.line['in_service'].copy()
trafo_backup = gridPP.trafo['in_service'].copy()

problematic_line_pairs = []
# 对每个重复的 line pair/group 单独测试
pairs = gridPP.line.groupby(['from_bus','to_bus']).apply(lambda df: list(df.index)).to_dict()
dup_pairs = {k:v for k,v in pairs.items() if len(v)>1}

for k, idx_list in dup_pairs.items():
    # disable all extras for this pair only, leave others as original
    gridPP.line['in_service'] = line_backup.copy()
    for idx in idx_list[1:]:
        gridPP.line.at[idx, 'in_service'] = False
    try:
        pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
        ok = True
    except Exception as e:
        ok = False
    print(f"Testing pair {k}, indices {idx_list} -> converged? {ok}")
    if not ok:
        problematic_line_pairs.append((k, idx_list))

# restore
gridPP.line['in_service'] = line_backup

# 然后对 trafo 的并联组做同样测试
problematic_trafo_pairs = []
trafo_pairs = gridPP.trafo.groupby(['hv_bus','lv_bus']).apply(lambda df: list(df.index)).to_dict()
dup_trafos = {k:v for k,v in trafo_pairs.items() if len(v)>1}

for k, idx_list in dup_trafos.items():
    gridPP.trafo['in_service'] = trafo_backup.copy()
    for idx in idx_list[1:]:
        gridPP.trafo.at[idx, 'in_service'] = False
    try:
        pp.runpp(gridPP, algorithm='nr', init='dc', max_iteration=200, tolerance_mva=1e-8)
        ok = True
    except Exception as e:
        ok = False
    print(f"Testing trafo group {k}, indices {idx_list} -> converged? {ok}")
    if not ok:
        problematic_trafo_pairs.append((k, idx_list))

# restore
gridPP.trafo['in_service'] = trafo_backup

print("Problematic line pairs (that STILL fail even when extras disabled):", problematic_line_pairs)
print("Problematic trafo groups:", problematic_trafo_pairs)

Testing pair (372, 418), indices [133, 233] -> converged? False
Testing pair (372, 655), indices [124, 208] -> converged? False
Testing pair (372, 704), indices [143, 227, 232] -> converged? False
Testing pair (392, 571), indices [136, 140] -> converged? False
Testing pair (392, 594), indices [134, 225] -> converged? False
Testing pair (418, 424), indices [181, 231] -> converged? False
Testing pair (424, 430), indices [127, 151] -> converged? False
Testing pair (430, 520), indices [125, 186] -> converged? False
Testing pair (473, 601), indices [129, 173] -> converged? False
Testing pair (473, 704), indices [122, 123, 226] -> converged? False
Testing pair (506, 526), indices [132, 187] -> converged? False
Testing pair (506, 645), indices [177, 178] -> converged? False
Testing pair (520, 526), indices [128, 170] -> converged? False
Testing pair (540, 594), indices [131, 139] -> converged? False
Testing pair (540, 601), indices [130, 166] -> converged? False
Testing pair (552, 630), indic